In [1]:
import sys
sys.path.append('..')

import pickle
from sklearn.metrics import (
    mean_squared_error, accuracy_score, log_loss, r2_score)
import pandas as pd
import numpy as np
from QHFactor.fn import *
from QHFactor.factors import HFTFactors
import warnings
from QHFactor.fn import (
    ts_winsorize,
    ts_zscore,
    delay,
    delta
)
from QHMlflow.estimator import LGBMRollingRegressor
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from QHMlflow.secuntial import Sequential
seq = Sequential(name='HFT')

warnings.filterwarnings('ignore')

Error importing QUANTAXIS module: No module named 'QUANTAXIS'


/opt/homebrew/Caskroom/miniconda/base/envs/QUANTHUB/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(
[2024-06-15 00:18:34 - INFO] Model name: HFT
[2024-06-15 00:18:34 - INFO] Training started at 2024-06-15 00:18:34


dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/QUANTHUB/lib/python3.10/site-packages/wtpy/wrapper/linux/libWtDtHelper.so, 0x0006): tried: '/opt/homebrew/Caskroom/miniconda/base/envs/QUANTHUB/lib/python3.10/site-packages/wtpy/wrapper/linux/libWtDtHelper.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Caskroom/miniconda/base/envs/QUANTHUB/lib/python3.10/site-packages/wtpy/wrapper/linux/libWtDtHelper.so' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/QUANTHUB/lib/python3.10/site-packages/wtpy/wrapper/linux/libWtDtHelper.so' (not a mach-o file), '/opt/homebrew/Caskroom/miniconda/base/envs/quanthub/lib/python3.10/site-packages/wtpy/wrapper/linux/libWtDtHelper.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Caskroom/miniconda/base/envs/quanthub/lib/python3.10/site-packages/wtpy/wrapper/linux/libWtDtHelper.so' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/quanthub/lib/python3.10/site-packages/wtpy/wrapper/li

In [ ]:
# 读取板底数据
df = pd.read_parquet('../QHData/pp2409.parquet')

last_price = df['price'].to_numpy()
ask_price = df['ask_price_0'].to_numpy()
bid_price = df['bid_price_0'].to_numpy()
bid_qty = df['bid_qty_0'].to_numpy()
ask_qty = df['bid_qty_0'].to_numpy()
volume = df['volume'].to_numpy()
turn_over = df['turn_over'].to_numpy()
open_interest = df['open_interest'].to_numpy()
diff_interest = df['diff_interest'].to_numpy()
total_turnover = df['total_turnover'].to_numpy()
total_volume = df['total_volume'].to_numpy()

In [ ]:
# 因子计算
window = 360

factors = pd.DataFrame(data={
    'price': last_price,
    'skew': ts_skew(last_price, window),
    'kurt': ts_kurt(last_price, window),
    'rwr': HFTFactors.rwr(last_price, window),
    'z_t': HFTFactors.z_t(last_price, ask_price, bid_price),
    # 'slpoe': HFTFactors.slpoe(bid_price, ask_price, bid_qty, ask_qty),
    'speculation': HFTFactors.speculation(volume, open_interest, window),
    'diff_interest': HFTFactors.diff_interest_sum(diff_interest, window),
    # 'voi': HFTFactors.voi(bid_price, ask_price, bid_qty, ask_qty, volume),
    'volatilaty': ts_stddev(last_price, window),
    "avgPx": HFTFactors.avg_price(total_turnover/5, total_volume, last_price),
    'mbp': HFTFactors.mbp(turn_over/5, volume, bid_price, ask_price),
    'positive_ratio': HFTFactors.positive_ratio(turn_over, last_price, ask_price, window),
    'negtive_ratio': HFTFactors.negtive_ratio(turn_over, last_price, bid_price, window),
}, index=df.index
)

factors['price'] = delay(ts_sum(delta(last_price,1),120),-120)

In [ ]:
factors = factors.transform(lambda x: ts_winsorize(
    x.to_numpy(), window)).dropna(how='any')

X, y = factors.drop('price', axis=1).transform(
    lambda x: ts_zscore(x.to_numpy(), window)), factors['price']

print(X.corrwith(y))

model = lgb.LGBMRegressor(importance_type='gain',
                          n_estimators=1000, random_state=2024)
model.fit(X, y)

title = 'LightGBM Feature Importance (gain)'
lgb.plot_importance(model,
                    importance_type='gain',
                    max_num_features=20,
                    figsize=(8, 6),
                    grid=False,
                    title=title
                    )

# %%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False)

# model = LGBMRollingRegressor().optimizer('HFT08', X_train, y_train)
model = seq.train_model(LGBMRollingRegressor,X_train,y_train)

model.predict(X_test)
y_pred = model.predict(X_train)
print(f'{np.percentile(y_pred,5)} {np.percentile(y_pred,95)}')


# -0.3247957456818994 0.23797592295645276 bu
# -0.36696193028613927 0.3064388737879785 fu
# -0.3349361096295464 0.2860071112663756 m
# -0.3481025644484865 0.25500206765461264 rm
# -0.35048536769769406 0.40468895580618897 v
# -0.3283572190749224 0.39061204862299276 pp

# 保存模型
pickle.dump(model,open('HFT_LGB_PP.pkl','wb'))